In [1]:
import numpy as np
import pandas as pd
import sys
from sklearn import ensemble as em
train = pd.read_csv('year1-data.csv')
train2  = pd.read_csv('year2-data.csv')
X_test = pd.read_csv('testing_data.csv')
#train.shape #6588(366*18)*26
#train2.shape #6570(365*18)*26
#X_test.shape #9000*11

In [2]:
#train = train.replace("#","")
#train = train.replace("\*","")
#train = train.replace("x","")
def preprocess(train):
    train.replace("#",'',inplace=True,regex=True)
    train.replace("x",'',inplace=True,regex=True)
    train.replace("\*",'',inplace=True,regex=True)
    train.replace("NR",0,inplace=True) #將NR變成0
    train.fillna(0, inplace=True) #把NAN變成0
    train = train.iloc[:,2:]
    return train

In [3]:
train =  preprocess(train) #6588*24
train2 = preprocess(train2) #6570*24
X_test = preprocess(X_test) #9000*9


In [4]:
def get_feature(train,test=False):
    train = np.array(train,dtype=float)# to array 6588*24
    tmp=[]
    feature = []
    for i  in range(train.shape[0]): #0~6587
        if not test:
            if i/18>180 and i/18<240:
                continue
        index = i%18
        if index==2 or index==4 or index==5 or index==7 or index==8 or index==9 or index==12:#只取CO,NO,NO2,PM2.5,PM10,O3,SO2
            tmp.append(train[i])
        if index==17:
            tmp = np.array(tmp,dtype=float)#shape7*24
            feature.append(tmp)
            tmp = []
    feature = np.array(feature,dtype=float)#366*168
    print(feature.shape)
    return feature

In [5]:
train = get_feature(train)#shape 366*168
train2 = get_feature(train2)#shape 365*168
X_test = get_feature(X_test,test=True)#500*7*9


(306, 7, 24)
(305, 7, 24)
(500, 7, 9)


In [6]:
train = np.concatenate((train, train2), axis=0)#shape 731*7*24

In [7]:
tmp = train[0]
tmp.shape
for i in range(1,train.shape[0]):
    tmp =  np.concatenate((tmp, train[i]), axis=1)
train = tmp #7*17544
x_mean = train.mean(axis=1,keepdims = True)
x_std = train.std(axis=1, keepdims = True)


In [8]:
def valid(x, y):
    if y <= 2 or y > 100:
        return False
    for i in range(9):
        if x[5,i] <= 2 or x[5,i] > 100:
            return False
    for i in range(9):
        for j in range(7):
            if x[j,i] < 0:
                return False
    return True

In [9]:
tmp = []
X_test.shape
for i in range(X_test.shape[0]):
    X_test[i] = (X_test[i]-x_mean)/x_std
    tmp.append(X_test[i].flatten())
X_test = tmp 
X_test = np.array(X_test,dtype=float)#shape 500*63

X_test.shape


(500, 63)

In [10]:
X=[]
Y=[]
for i in range(0, train.shape[1]-9):
    x =  train[:, i:i+9]
    y = train[5, i+9]
    if valid(x,y):
        x = np.array(x)
        x =(x-x_mean)/x_std
        X.append(x.flatten())
        Y.append([y])

X = np.array(X)#shape 5845*63
Y = np.array(Y)#shape 5845*1
y_mean = Y.mean()
y_std = Y.std()
Y = (Y-y_mean)/y_std



#X_1=X[0:2327,:]
#X_2=X[2327:4654,:]
#X_3=X[4654:6981,:]
#X_4=X[6981:9308,:]
#X_5=X[9308:,:]
#Y_1 = Y[0:2327,:]
#Y_2 = Y[2327:4654,:]
#Y_3 = Y[4654:6981,:]
#Y_4 = Y[6981:9308,:]
#Y_5 = Y[9308:,:]


In [11]:
class Linear_Regression():
    def __init__(self,B = np.double(0),W = np.ones((63, 1))):
        self.B = B
        self.W = W
        pass
    def error(self, X, Y):
        return Y - self.predict(X)

    def MSE(self, X, Y):#MSE(X) = E[(X-Y)^2]
        return np.sqrt(np.mean(self.error(X, Y) ** 2))
    
    def predict(self, X):#   y = wx+b
        _X = np.reshape(X, (-1, self.feature_dim))
        return np.dot(_X, self.W) + self.B

    def fit(self, _X, Y, valid, max_epoch=500000, lr=0.000001, C=0.0):
        assert _X.shape[0] == Y.shape[0]
        N = _X.shape[0]
        self.feature_dim = feature_dim = _X.shape[1]
        self.C = C
        print(_X.shape)
        X = _X
        
     
        #self.B = 0.0
        #self.W = np.ones((self.feature_dim, 1))
        for epoch in range(1, max_epoch+1):
            
            error = Y - self.predict(X)          
            B_grad = -np.sum(error)  / N
            W_grad = -np.dot(X.T, error) / N
            self.B = self.B - lr  * B_grad
            self.W = self.W - lr  * W_grad
            
            if epoch % 1000 == 0:
                print('[Epoch {}]: loss: {}'.format(epoch, self.MSE(X, Y)))


In [12]:
model = Linear_Regression()
model.fit(X, Y, valid=None, max_epoch=800000, lr=0.000003, C=0.00001)

(11635, 63)
[Epoch 1000]: loss: 23.714122883451363
[Epoch 2000]: loss: 22.77576967179622
[Epoch 3000]: loss: 21.882880099190594
[Epoch 4000]: loss: 21.03319321976658
[Epoch 5000]: loss: 20.224556139980006
[Epoch 6000]: loss: 19.454918919061218
[Epoch 7000]: loss: 18.722329728934408
[Epoch 8000]: loss: 18.024930260405736
[Epoch 9000]: loss: 17.360951362881174
[Epoch 10000]: loss: 16.72870890531714
[Epoch 11000]: loss: 16.12659984653444
[Epoch 12000]: loss: 15.553098503443616
[Epoch 13000]: loss: 15.006753006143462
[Epoch 14000]: loss: 14.486181929265575
[Epoch 15000]: loss: 13.990071089352956
[Epoch 16000]: loss: 13.517170498480109
[Epoch 17000]: loss: 13.06629146474965
[Epoch 18000]: loss: 12.6363038307366
[Epoch 19000]: loss: 12.226133341397045
[Epoch 20000]: loss: 11.834759133412208
[Epoch 21000]: loss: 11.461211338401133
[Epoch 22000]: loss: 11.104568792902963
[Epoch 23000]: loss: 10.763956848500465
[Epoch 24000]: loss: 10.438545275927378
[Epoch 25000]: loss: 10.127546257468811
[Epo

[Epoch 206000]: loss: 1.0749522610457456
[Epoch 207000]: loss: 1.0706748787715852
[Epoch 208000]: loss: 1.0664569819648664
[Epoch 209000]: loss: 1.062297428297922
[Epoch 210000]: loss: 1.0581951014613402
[Epoch 211000]: loss: 1.0541489105038369
[Epoch 212000]: loss: 1.050157789189872
[Epoch 213000]: loss: 1.0462206953745479
[Epoch 214000]: loss: 1.042336610395328
[Epoch 215000]: loss: 1.0385045384801301
[Epoch 216000]: loss: 1.0347235061713587
[Epoch 217000]: loss: 1.0309925617654474
[Epoch 218000]: loss: 1.0273107747674934
[Epoch 219000]: loss: 1.023677235360581
[Epoch 220000]: loss: 1.0200910538893957
[Epoch 221000]: loss: 1.0165513603577405
[Epoch 222000]: loss: 1.0130573039395743
[Epoch 223000]: loss: 1.0096080525032072
[Epoch 224000]: loss: 1.0062027921482903
[Epoch 225000]: loss: 1.002840726755251
[Epoch 226000]: loss: 0.9995210775468295
[Epoch 227000]: loss: 0.9962430826613831
[Epoch 228000]: loss: 0.9930059967376393
[Epoch 229000]: loss: 0.9898090905105698
[Epoch 230000]: loss:

[Epoch 407000]: loss: 0.7207420836745934
[Epoch 408000]: loss: 0.7200108772773708
[Epoch 409000]: loss: 0.7192837595177037
[Epoch 410000]: loss: 0.7185606960362482
[Epoch 411000]: loss: 0.717841652894557
[Epoch 412000]: loss: 0.7171265965683299
[Epoch 413000]: loss: 0.7164154939407854
[Epoch 414000]: loss: 0.7157083122961598
[Epoch 415000]: loss: 0.715005019313323
[Epoch 416000]: loss: 0.7143055830595153
[Epoch 417000]: loss: 0.713609971984196
[Epoch 418000]: loss: 0.7129181549130085
[Epoch 419000]: loss: 0.7122301010418561
[Epoch 420000]: loss: 0.7115457799310855
[Epoch 421000]: loss: 0.7108651614997773
[Epoch 422000]: loss: 0.7101882160201407
[Epoch 423000]: loss: 0.7095149141120122
[Epoch 424000]: loss: 0.7088452267374526
[Epoch 425000]: loss: 0.7081791251954439
[Epoch 426000]: loss: 0.7075165811166823
[Epoch 427000]: loss: 0.7068575664584653
[Epoch 428000]: loss: 0.7062020534996724
[Epoch 429000]: loss: 0.705550014835835
[Epoch 430000]: loss: 0.704901423374299
[Epoch 431000]: loss:

[Epoch 608000]: loss: 0.6271124857102843
[Epoch 609000]: loss: 0.6268214048253048
[Epoch 610000]: loss: 0.6265314495269372
[Epoch 611000]: loss: 0.6262426139285684
[Epoch 612000]: loss: 0.6259548921815654
[Epoch 613000]: loss: 0.6256682784749565
[Epoch 614000]: loss: 0.6253827670351195
[Epoch 615000]: loss: 0.6250983521254713
[Epoch 616000]: loss: 0.6248150280461622
[Epoch 617000]: loss: 0.6245327891337721
[Epoch 618000]: loss: 0.6242516297610128
[Epoch 619000]: loss: 0.6239715443364292
[Epoch 620000]: loss: 0.6236925273041092
[Epoch 621000]: loss: 0.6234145731433913
[Epoch 622000]: loss: 0.6231376763685808
[Epoch 623000]: loss: 0.6228618315286636
[Epoch 624000]: loss: 0.6225870332070285
[Epoch 625000]: loss: 0.6223132760211877
[Epoch 626000]: loss: 0.6220405546225044
[Epoch 627000]: loss: 0.6217688636959209
[Epoch 628000]: loss: 0.6214981979596886
[Epoch 629000]: loss: 0.6212285521651052
[Epoch 630000]: loss: 0.6209599210962501
[Epoch 631000]: loss: 0.6206922995697249
[Epoch 632000]: 

In [13]:
predict = model.predict(X_test)
print(predict.shape)
for i in range(len(predict)):
    predict[i] = (predict[i]*y_std)+y_mean
    if predict[i]< 0:
        predict[i] = 4

(500, 1)


In [14]:
 with open('line_regression_normal_result.csv', 'w') as f:
        print('id,value', file=f)
        for (i, p) in enumerate(predict) :
            print('id_{},{}'.format(i, p[0]), file=f)

# TODO #

* ~~去掉七八月~~
* ~~shift 3 小時~~
* ~~只做PM2.5 PM10 NO2影響大......~~
* ~~負的資料砍掉~~
* n-fold
* normalize
* gradient bootsing

